In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
2315119,2022-04-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2315120,2022-04-16,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2315121,2022-04-17,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2315122,2022-04-18,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
59467,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
59469,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
59471,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
59473,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
59475,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1401695,2022-04-15,Arkansas,Arkansas,5001,5682,79.00,5000,Arkansas,AR,Arkansas,State,3017804
1401697,2022-04-16,Arkansas,Arkansas,5001,5681,79.00,5000,Arkansas,AR,Arkansas,State,3017804
1401699,2022-04-17,Arkansas,Arkansas,5001,5681,79.00,5000,Arkansas,AR,Arkansas,State,3017804
1401701,2022-04-18,Arkansas,Arkansas,5001,5681,79.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2315119,2022-04-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2315120,2022-04-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2315121,2022-04-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2315122,2022-04-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2022-04-19') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
819,2022-04-19,Snohomish,Washington,152500,1129.00,53061,WA,County,822083,18550.44,137.33
1636,2022-04-19,Cook,Illinois,1144935,14869.00,17031,IL,County,5150233,22230.74,288.71
2452,2022-04-19,Orange,California,590828,6966.00,6059,CA,County,3175692,18604.70,219.35
3267,2022-04-19,Maricopa,Arizona,1273651,17079.00,4013,AZ,County,4485414,28395.39,380.77
4082,2022-04-19,Los Angeles,California,2855271,31884.00,6037,CA,County,10039107,28441.48,317.60
...,...,...,...,...,...,...,...,...,...,...,...
2313031,2022-04-19,Wheeler,Oregon,243,3.00,41069,OR,County,1332,18243.24,225.23
2313585,2022-04-19,King,Texas,51,0.00,48269,TX,County,272,18750.00,0.00
2314108,2022-04-19,Esmeralda,Nevada,109,3.00,32009,NV,County,873,12485.68,343.64
2314627,2022-04-19,Loving,Texas,185,1.00,48301,TX,County,169,109467.46,591.72


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
819,2022-04-19,Snohomish,Washington,152500,1129.00,53061,WA,County,822083,18550.44,137.33,137.33,18550.44
1636,2022-04-19,Cook,Illinois,1144935,14869.00,17031,IL,County,5150233,22230.74,288.71,288.71,22230.74
2452,2022-04-19,Orange,California,590828,6966.00,6059,CA,County,3175692,18604.70,219.35,219.35,18604.70
3267,2022-04-19,Maricopa,Arizona,1273651,17079.00,4013,AZ,County,4485414,28395.39,380.77,380.77,28395.39
4082,2022-04-19,Los Angeles,California,2855271,31884.00,6037,CA,County,10039107,28441.48,317.60,317.60,28441.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2313031,2022-04-19,Wheeler,Oregon,243,3.00,41069,OR,County,1332,18243.24,225.23,225.23,18243.24
2313585,2022-04-19,King,Texas,51,0.00,48269,TX,County,272,18750.00,0.00,0.00,18750.00
2314108,2022-04-19,Esmeralda,Nevada,109,3.00,32009,NV,County,873,12485.68,343.64,343.64,12485.68
2314627,2022-04-19,Loving,Texas,185,1.00,48301,TX,County,169,109467.46,591.72,591.72,109467.46


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1466372,2022-04-19,Hale,Alabama,4722,106.00,1065,AL,County,14651,32229.88,723.50,723.50,32229.88,1
1236686,2022-04-19,Winston,Alabama,7569,130.00,1133,AL,County,23629,32032.67,550.17,550.17,32032.67,2
849456,2022-04-19,Franklin,Alabama,9864,138.00,1059,AL,County,31362,31452.08,440.02,440.02,31452.08,3
1010775,2022-04-19,Clay,Alabama,4082,81.00,1027,AL,County,13235,30842.46,612.01,612.01,30842.46,4
1465621,2022-04-19,Fayette,Alabama,4983,96.00,1057,AL,County,16302,30566.80,588.88,588.88,30566.80,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980468,2022-04-19,Big Horn,Wyoming,2580,60.00,56003,WY,County,11790,21882.95,508.91,508.91,21882.95,19
1232916,2022-04-19,Johnson,Wyoming,1845,19.00,56019,WY,County,8445,21847.25,224.99,224.99,21847.25,20
1743579,2022-04-19,Lincoln,Wyoming,3972,31.00,56023,WY,County,19830,20030.26,156.33,156.33,20030.26,21
1399447,2022-04-19,Sublette,Wyoming,1922,27.00,56035,WY,County,9831,19550.40,274.64,274.64,19550.40,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
1144581,2022-04-19,Lowndes,Alabama,2611,76.00,1085,AL,County,9726,26845.57,781.41,781.41,26845.57,31,1
1466372,2022-04-19,Hale,Alabama,4722,106.00,1065,AL,County,14651,32229.88,723.50,723.50,32229.88,1,2
1143071,2022-04-19,Crenshaw,Alabama,3802,99.00,1041,AL,County,13772,27606.74,718.85,718.85,27606.74,25,3
483616,2022-04-19,Walker,Alabama,18726,443.00,1127,AL,County,63521,29480.01,697.41,697.41,29480.01,9,4
1008507,2022-04-19,Butler,Alabama,5059,129.00,1013,AL,County,19448,26012.96,663.31,663.31,26012.96,37,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232916,2022-04-19,Johnson,Wyoming,1845,19.00,56019,WY,County,8445,21847.25,224.99,224.99,21847.25,20,19
1699506,2022-04-19,Uinta,Wyoming,5625,39.00,56041,WY,County,20226,27810.74,192.82,192.82,27810.74,7,20
1743579,2022-04-19,Lincoln,Wyoming,3972,31.00,56023,WY,County,19830,20030.26,156.33,156.33,20030.26,21,21
1140051,2022-04-19,Albany,Wyoming,9894,48.00,56001,WY,County,38880,25447.53,123.46,123.46,25447.53,14,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,137.33,18550.44,30,17
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,137.33,18550.44,30,17
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,137.33,18550.44,30,17
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,137.33,18550.44,30,17
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,137.33,18550.44,30,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309785,2022-04-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2309786,2022-04-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2309787,2022-04-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2309788,2022-04-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1460288,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,32229.88,2,1,1.00,nan
1460289,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,32229.88,2,1,0.00,nan
1460290,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,32229.88,2,1,0.00,nan
1460291,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,723.50,32229.88,2,1,0.00,nan
1460292,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,723.50,32229.88,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773303,2022-04-15,Crook,Wyoming,1346,28.00,56011,WY,County,7584,17747.89,369.20,369.20,17761.08,9,23,0.00,0.00
1773304,2022-04-16,Crook,Wyoming,1346,28.00,56011,WY,County,7584,17747.89,369.20,369.20,17761.08,9,23,0.00,0.00
1773305,2022-04-17,Crook,Wyoming,1346,28.00,56011,WY,County,7584,17747.89,369.20,369.20,17761.08,9,23,0.00,0.00
1773306,2022-04-18,Crook,Wyoming,1346,28.00,56011,WY,County,7584,17747.89,369.20,369.20,17761.08,9,23,0.00,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1139999,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,781.41,26845.57,1,31,1.00,0.21,0.00,nan
1140000,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,781.41,26845.57,1,31,0.00,0.21,0.00,nan
1140001,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,781.41,26845.57,1,31,0.00,0.21,0.00,nan
1140002,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,781.41,26845.57,1,31,0.00,0.21,0.00,nan
1140003,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,781.41,26845.57,1,31,0.00,0.21,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478254,2022-04-15,Teton,Wyoming,9945,16.00,56039,WY,County,23464,42384.08,68.19,68.19,42511.93,23,1,0.00,1.93,0.00,0.00
478255,2022-04-16,Teton,Wyoming,9945,16.00,56039,WY,County,23464,42384.08,68.19,68.19,42511.93,23,1,0.00,1.93,0.00,0.00
478256,2022-04-17,Teton,Wyoming,9945,16.00,56039,WY,County,23464,42384.08,68.19,68.19,42511.93,23,1,0.00,1.93,0.00,0.00
478257,2022-04-18,Teton,Wyoming,9945,16.00,56039,WY,County,23464,42384.08,68.19,68.19,42511.93,23,1,0.00,1.93,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2022-04-19') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
578072,2022-04-19,Imperial,California,66723,909.00,6025,CA,County,181215,36819.80,501.61,501.61,36819.80,1,1,34.00,21.50,0.00,0.14
88082,2022-04-19,Shasta,California,36771,621.00,6089,CA,County,180080,20419.26,344.85,344.85,20419.26,2,28,4.00,1.79,0.00,0.79
1400129,2022-04-19,Tuolumne,California,13493,181.00,6109,CA,County,54478,24767.80,332.24,332.24,24767.80,3,14,0.00,2.14,0.00,0.00
286446,2022-04-19,San Bernardino,California,590227,6977.00,6071,CA,County,2180085,27073.58,320.03,320.03,27073.58,4,7,1008.00,278.86,29.00,6.71
4082,2022-04-19,Los Angeles,California,2855271,31884.00,6037,CA,County,10039107,28441.48,317.60,317.60,28441.48,5,5,861.00,1224.07,12.00,11.57
1016775,2022-04-19,Inyo,California,4569,56.00,6027,CA,County,18039,25328.46,310.44,310.44,25328.46,6,11,0.00,1.21,0.00,0.00
158214,2022-04-19,Tulare,California,134546,1439.00,6107,CA,County,466195,28860.46,308.67,308.67,28860.46,7,4,217.00,91.93,4.00,1.79
127417,2022-04-19,Stanislaus,California,136645,1694.00,6099,CA,County,550660,24814.77,307.63,307.63,24814.77,8,13,0.00,0.00,0.00,0.00
1698654,2022-04-19,Tehama,California,14690,199.00,6103,CA,County,65084,22570.83,305.76,305.76,22570.83,9,20,0.00,2.50,0.00,0.07
920653,2022-04-19,Merced,California,72311,818.00,6047,CA,County,277680,26041.13,294.58,294.58,26041.13,10,9,15.00,19.50,0.00,0.21


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
578072,2022-04-19,Imperial,California,66723,909.00,6025,CA,County,181215,36819.80,501.61,501.61,36819.80,1,1,34.00,21.50,0.00,0.14
1235120,2022-04-19,Kings,California,55198,445.00,6031,CA,County,152940,36091.28,290.96,290.96,36091.28,12,2,27.00,5.93,0.00,0.14
2177341,2022-04-19,Lassen,California,10723,69.00,6035,CA,County,30573,35073.43,225.69,225.69,35073.43,20,3,0.00,0.43,0.00,0.00
158214,2022-04-19,Tulare,California,134546,1439.00,6107,CA,County,466195,28860.46,308.67,308.67,28860.46,7,4,217.00,91.93,4.00,1.79
4082,2022-04-19,Los Angeles,California,2855271,31884.00,6037,CA,County,10039107,28441.48,317.60,317.60,28441.48,5,5,861.00,1224.07,12.00,11.57
57918,2022-04-19,Madera,California,43445,354.00,6039,CA,County,157327,27614.46,225.01,225.01,27614.46,21,6,0.00,2.07,0.00,0.00
286446,2022-04-19,San Bernardino,California,590227,6977.00,6071,CA,County,2180085,27073.58,320.03,320.03,27073.58,4,7,1008.00,278.86,29.00,6.71
360693,2022-04-19,Kern,California,243003,2344.00,6029,CA,County,900202,26994.27,260.39,260.39,26994.27,15,8,115.00,28.93,10.00,2.86
920653,2022-04-19,Merced,California,72311,818.00,6047,CA,County,277680,26041.13,294.58,294.58,26041.13,10,9,15.00,19.50,0.00,0.21
57144,2022-04-19,Fresno,California,254703,2740.00,6019,CA,County,999101,25493.22,274.25,274.25,25493.22,13,10,113.00,73.36,1.00,1.07


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
578072,2022-04-19,Imperial,California,66723,909.00,6025,CA,County,181215,36819.80,501.61,501.61,36819.80,1,1,34.00,21.50,0.00,0.14
88082,2022-04-19,Shasta,California,36771,621.00,6089,CA,County,180080,20419.26,344.85,344.85,20419.26,2,28,4.00,1.79,0.00,0.79
1400129,2022-04-19,Tuolumne,California,13493,181.00,6109,CA,County,54478,24767.80,332.24,332.24,24767.80,3,14,0.00,2.14,0.00,0.00
286446,2022-04-19,San Bernardino,California,590227,6977.00,6071,CA,County,2180085,27073.58,320.03,320.03,27073.58,4,7,1008.00,278.86,29.00,6.71
4082,2022-04-19,Los Angeles,California,2855271,31884.00,6037,CA,County,10039107,28441.48,317.60,317.60,28441.48,5,5,861.00,1224.07,12.00,11.57
1016775,2022-04-19,Inyo,California,4569,56.00,6027,CA,County,18039,25328.46,310.44,310.44,25328.46,6,11,0.00,1.21,0.00,0.00
158214,2022-04-19,Tulare,California,134546,1439.00,6107,CA,County,466195,28860.46,308.67,308.67,28860.46,7,4,217.00,91.93,4.00,1.79
127417,2022-04-19,Stanislaus,California,136645,1694.00,6099,CA,County,550660,24814.77,307.63,307.63,24814.77,8,13,0.00,0.00,0.00,0.00
1698654,2022-04-19,Tehama,California,14690,199.00,6103,CA,County,65084,22570.83,305.76,305.76,22570.83,9,20,0.00,2.50,0.00,0.07
920653,2022-04-19,Merced,California,72311,818.00,6047,CA,County,277680,26041.13,294.58,294.58,26041.13,10,9,15.00,19.50,0.00,0.21


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,4.00,0.36
1,03/21/20,Imperial,4,2.21,0.00,0.36
2,03/22/20,Imperial,4,2.21,0.00,0.36
3,03/23/20,Imperial,4,2.21,0.00,0.36
4,03/24/20,Imperial,9,4.97,5.00,0.71
...,...,...,...,...,...,...
7626,04/15/22,Madera,43445,27614.46,1.00,3.21
7627,04/16/22,Madera,43445,27614.46,0.00,3.21
7628,04/17/22,Madera,43445,27614.46,0.00,3.21
7629,04/18/22,Madera,43445,27614.46,0.00,3.21


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,0.00,0.00
1,03/21/20,Imperial,0.00,0.00,0.00,0.00
2,03/22/20,Imperial,0.00,0.00,0.00,0.00
3,03/23/20,Imperial,0.00,0.00,0.00,0.00
4,03/24/20,Imperial,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...
7660,04/15/22,Merced,818.00,294.58,0.00,0.36
7661,04/16/22,Merced,818.00,294.58,0.00,0.36
7662,04/17/22,Merced,818.00,294.58,0.00,0.36
7663,04/18/22,Merced,818.00,294.58,0.00,0.36


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)